Implementazione di un filtro di Kalman

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def predict(A, B, Q, u_t, mu_t, Sigma_t):
    predicted_mu = A @ mu_t + B @ u_t
    predicted_Sigma = A @ Sigma_t @ A.T + Q
    return predicted_mu, predicted_Sigma


def update(H, R, z, predicted_mu, predicted_Sigma):
    residual_mean = z - H @ predicted_mu
    residual_covariance = H @ predicted_Sigma @ H.T + R
    kalman_gain = predicted_Sigma @ H.T @ np.linalg.inv(residual_covariance)
    updated_mu = predicted_mu + kalman_gain @ residual_mean
    updated_Sigma = predicted_Sigma - kalman_gain @ H @ predicted_Sigma
    return updated_mu, updated_Sigma

In [15]:
#Physical constant
m = 0.2 # mass of the blimp
Fl = 5/1000 * 9.81 #force of 5 grams of l motor 
Fr = 5/1000 * 9.81 #force of 5 grams of r motor 
Fu = 5/1000 * 9.81 #force of 5 grams of u motor 
psi = 0 #measured by the sensor

# Initialize the problem with the given information
mu_0 = np.array([0, 0, 0])
Sigma_0 = np.array([[0.1, 0, 0],
                     [0, 0.1, 0],
                     [0, 0, 0.1]])


num_steps = 100
ground_truth_xs = np.linspace(0, 10, num=num_steps + 1) # [0, 1, ..., 10]
ground_truth_ys = ground_truth_xs.copy() # x = y
ground_truth_zs = ground_truth_xs.copy() # x = y

ground_truth_states = np.stack((ground_truth_xs,ground_truth_ys, ground_truth_zs ), axis=1) # ground_truth_states is [[0,0], [1,1], ..., [10,10]]


# State = [x_pos, y_pos, z_pos]

# Initial position
x_0, y_0, z_0 = 0, 0, 0  #given initial position
motion_states  = [np.array([x_0, y_0, z_0])] # a list to store state at each step following noisy motion model
u_t = np.array([Fl/m, Fr/m, Fu/m]) #input vector depending on force

A = np.array([[1, 0, 0],
              [0, 1, 0],
              [0, 0, 1]])

B = np.array([[np.cos(psi), np.cos(psi), 0],
               [np.sin(psi), np.sin(psi), 0],
               [0, 0, 1] ])

Q = np.array([[0.01, 0, 0], #Covariance matrix of state estimation
              [0, 0.01, 0], #error
              [0, 0, 0.01]])

H = np.array([[1, 0, 0],  #Measure of the Sensors:
              [0, 1, 0],  # UWB + Ultrasonic
              [0, 0, 1]])

R = np.array([[0.75, 0, 0], #Error of the sensors
              [0, 0.75, 0],
              [0, 0, 1]])


 # Initialize empty lists to store the filtered states and measurements for plotting
measurement_states = []
filtered_states = [] 

# Run KF for each time step
mu_current = mu_0.copy()
Sigma_current = Sigma_0.copy()
i = 0
while i< 100: 
    # Predict step
    predicted_mu, predicted_Sigma = predict(A, B, Q, u_t, mu_current, Sigma_current)
    
    # Get measurement (in real life, we get this from our sensor)    
    measurement_noise = np.random.multivariate_normal(mean=np.array([0,0,0]), cov=R) # ~N(0,R)
    new_measurement = H @ ground_truth_states[i+1] + measurement_noise # this is z_t
    
    # The rest of update step
    mu_current, Sigma_current = update(H, R, new_measurement, predicted_mu, predicted_Sigma)
    
    # Store measurements and mu_current so we can plot it later
    measurement_states.append(new_measurement)
    filtered_states.append(mu_current)
    i += 1



    

